# Diplib

In [26]:
import diplib as dip
import os
import pandas as pd
from tqdm import tqdm
# Directory containing the images and where to save cleaned images
image_folder = 'Camera2/front'

# Load the background image
background_image_path = 'Main.jpeg'
bg = dip.ImageRead(background_image_path)

# Prepare a DataFrame to store results
results_df = pd.DataFrame(columns=['Filename', 'MaxFeret', 'MinFeret', 'SolidArea','Perimeter', 'Radius'])

# Iterate through each file in the folder
for filename in tqdm(os.listdir(image_folder)):
    if filename.endswith('.jpeg') or filename.endswith('.jpg'):
        image_path = os.path.join(image_folder, filename)
        
        try:
            # Load the current image
            img = dip.ImageRead(image_path)
            
            # Calculate difference and filter
            diff = dip.MaximumTensorElement(dip.Abs(img - bg))
            diff = dip.MedianFilter(diff)
            
            objects = diff > 15  # Apply the Otsu threshold
            
            objects = dip.Label(objects, mode='largest')  # the labeled image will have only one object in it
            msr = dip.MeasurementTool.Measure(objects, features=['Feret'])
            max_len, min_len = msr[1]['Feret'][0], msr[1]['Feret'][1]  # Max and Min Feret diameters
    
            # Measure SolidArea
            solid_area_msr = dip.MeasurementTool.Measure(objects, features=['SolidArea'])
            solid_area = solid_area_msr[1]['SolidArea'][0]  # Solid area value

                        # Measure SolidArea
            per_msr = dip.MeasurementTool.Measure(objects, features=['Perimeter'])
            per = per_msr[1]['Perimeter'][0]  # Solid area value
            rad_msr=dip.MeasurementTool.Measure(objects, features=['Radius'])
            rad = rad_msr[1]['Radius'][0]
            # Extract the numerical part of the filename
            file_number = os.path.splitext(filename)[0]
    
            # Append the results to the DataFrame
            results_df = pd.concat([results_df, pd.DataFrame({
                'Filename': [file_number],
                'MaxFeret': [max_len],
                'MinFeret': [min_len],
                'SolidArea': [solid_area],
                'Perimeter':[per],
                'Radius':[rad]
            })], ignore_index=True)
    
        except Exception as e:
            print(f"Error processing image {filename}: {e}")

# Save the results to a CSV file if needed
results_df.to_csv('measurement_results2.csv', index=False)

print("Processing complete.")

  0%|                                                                                         | 0/1091 [00:00<?, ?it/s]C:\Users\adminsyma\AppData\Local\Temp\ipykernel_11328\932502492.py:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({
100%|██████████████████████████████████████████████████████████████████████████████| 1091/1091 [19:17<00:00,  1.06s/it]

Processing complete.


In [27]:
results_df

,Filename,MaxFeret,MinFeret,SolidArea,Perimeter,Radius
0,0,849.944116,303.629308,237298.0,2180.456593,425.710240
1,1,2171.427641,590.495572,933689.0,5312.106593,1100.568489
2,10,740.708445,508.430858,276391.0,2357.340593,371.311066
3,100,1383.974349,374.286618,495083.0,3374.926593,692.326804
4,1000,1658.334707,251.668815,400767.0,3791.250593,832.565524
...,...,...,...,...,...,...
1086,995,3007.541355,1267.783578,458709.0,9000.563593,1670.407214
1087,996,1304.436277,744.103397,777379.0,3851.577593,671.866359
1088,997,1300.629079,726.041100,777532.0,3746.687593,670.915349
1089,998,2048.109860,548.370646,1065927.0,5345.901593,1027.118212


In [28]:
849.944116 *303.629308

258067.94377975172

In [29]:
# Convert Perimeter from pixels to centimeters
results_df['MaxFeret(cm)'] = results_df['MaxFeret'] * 0.04

# Convert MaxFeretLength from pixels to centimeters
results_df['MinFeret(cm)'] = results_df['MinFeret'] * 0.04

# Convert MinFeretLength from pixels to centimeters
results_df['SolidArea(cm)'] = results_df['SolidArea'] * 0.04
# Convert SolidArea from pixels to centimeters
results_df['Perimeter(cm)'] = results_df['Perimeter'] * 0.04
# Convert SolidArea from pixels to centimeters
results_df['Radius(cm)'] = results_df['Radius'] * 0.04

In [43]:
results_df['Filename'] = results_df['Filename'].astype(int)


In [44]:
results_df = results_df.sort_values(by='Filename')

In [45]:
results_df

,Filename,MaxFeret,MinFeret,SolidArea,Perimeter,Radius,MaxFeret(cm),MinFeret(cm),SolidArea(cm),Perimeter(cm),Radius(cm)
0,0,849.944116,303.629308,237298.0,2180.456593,425.710240,33.997765,12.145172,9491.92,87.218264,17.028410
1,1,2171.427641,590.495572,933689.0,5312.106593,1100.568489,86.857106,23.619823,37347.56,212.484264,44.022740
210,2,2070.076327,1161.548818,1512373.0,5779.372593,1080.109099,82.803053,46.461953,60494.92,231.174904,43.204364
318,3,1770.647622,438.941628,694904.0,4282.618593,888.519227,70.825905,17.557665,27796.16,171.304744,35.540769
429,4,1850.213501,188.620739,331585.0,4018.092593,930.100915,74.008540,7.544830,13263.40,160.723704,37.204037
...,...,...,...,...,...,...,...,...,...,...,...
109,1096,2850.992810,144.168005,311761.0,6138.114593,1441.882879,114.039712,5.766720,12470.44,245.524584,57.675315
110,1097,931.919524,405.154308,137418.0,4557.047593,583.120165,37.276781,16.206172,5496.72,182.281904,23.324807
111,1098,2440.641104,1519.722508,1438735.0,14245.235593,1407.849778,97.625644,60.788900,57549.40,569.809424,56.313991
112,1099,2015.850193,124.640486,173041.0,4633.814593,1019.168600,80.634008,4.985619,6921.64,185.352584,40.766744


# OpenCV

In [38]:
import cv2
import numpy as np

def measure_radius(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold to create a binary image
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume the largest contour is the object
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the area of the largest contour
    area = cv2.contourArea(largest_contour)
    
    # Calculate the equivalent radius assuming a circle
    equivalent_radius = np.sqrt(area / np.pi)
    
    return equivalent_radius

# Example usage
image_path = 'CleanedOtsu/3_processed.jpeg'
radius = measure_radius(image_path)
print(f"The equivalent radius of the object is: {radius}")


The equivalent radius of the object is: 470.55406860183194


In [39]:
276* 0.04

11.040000000000001

Weird results

In [40]:
import cv2
import numpy as np

def measure_radius_and_feret(image_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply a binary threshold to create a binary image
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume the largest contour is the object
    largest_contour = max(contours, key=cv2.contourArea)
    
    # Calculate the area of the largest contour
    area = cv2.contourArea(largest_contour)
    
    # Calculate the equivalent radius assuming a circle
    equivalent_radius = np.sqrt(area / np.pi)
    
    # Calculate the Feret diameters
    # Create a rotated bounding box
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    
    # Calculate the distances between all pairs of box points
    distances = [np.linalg.norm(box[i] - box[j]) for i in range(4) for j in range(i + 1, 4)]
    
    # Maximum Feret diameter (max distance)
    max_feret = max(distances)
    
    # Minimum Feret diameter (min distance)
    min_feret = min(distances)
    
    return equivalent_radius, max_feret, min_feret

# Example usage
radius, max_feret, min_feret = measure_radius_and_feret(image_path)
print(f"Equivalent radius of the object: {radius}")
print(f"Maximum Feret diameter: {max_feret}")
print(f"Minimum Feret diameter: {min_feret}")


Equivalent radius of the object: 470.55406860183194
Maximum Feret diameter: 1784.8789874946704
Minimum Feret diameter: 447.0


C:\Users\adminsyma\AppData\Local\Temp\ipykernel_11328\1567720161.py:30: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
